In [1]:
from dotenv import load_dotenv
import datetime
from langchain_openai import ChatOpenAI

# Get the current date
current_date = datetime.datetime.now().date()
load_dotenv()
# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate


In [9]:
from langchain.chains import SequentialChain

In [11]:


from langchain.chains.llm import LLMChain

llm = ChatOpenAI(temperature=0.1, model=llm_model)
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

# prompt template 4: translate to english
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_messsage")

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "language", "followup_messsage"]
)

/Users/sonph36/dev/MAS_Applications/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
overall_chain(review)

/Users/sonph36/dev/MAS_Applications/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and quality of the product, suspecting that it may be an old batch or counterfeit.',
 'language': 'French',
 'followup_messsage': "Cher client,\n\nNous sommes désolés d'apprendre que vous n'êtes pas satisfait de notre produit. Nous prenons la qualité de nos produits très au sérieux et nous vous assurons que nous n'utilisons que des ingrédients de la plus haute qualité. Il est possible que vous ayez reçu un lot périmé ou contrefait, ce qui n'est pas du tout représentatif de notre norme de qualité.\n\nNous aimerions avoir plus d'informations sur votre expérience afin de pouvoir résoudre 

In [13]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [14]:
prompt_infos = [
    {
        "name": 'physics',
        "description": "Good at answer questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": 'math',
        "description": "Good at answer questions about math",
        "prompt_template": math_template
    },
    {
        "name": 'history',
        "description": "Good at answer questions about history",
        "prompt_template": history_template
    },
    {
        "name": 'computerscience',
        "description": "Good at answer questions about computer science",
        "prompt_template": computerscience_template
    },
]

In [15]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [16]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [18]:
destionation_chains = {}
for p_info in prompt_infos:
    name = p_info['name']
    prompt = ChatPromptTemplate.from_template(p_info['prompt_template'])
    chain = LLMChain(llm=llm, prompt=prompt)
    destionation_chains[name] = chain

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = '\n'.join(destinations)
print(destinations_str)

physics:Good at answer questions about physics
math:Good at answer questions about math
history:Good at answer questions about history
computerscience:Good at answer questions about computer science


In [19]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [23]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
Given a raw text input to a 
language model select the model prompt best suited for the input. 
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 
You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \\ name of the prompt to use or "DEFAULT"
    "next_inputs": string \\ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt 
names specified below OR it can be "DEFAULT" if the input is not
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input 
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [25]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template)


Given a raw text input to a 
language model select the model prompt best suited for the input. 
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 
You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt 
names specified below OR it can be "DEFAULT" if the input is not
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input 
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics:Good at answer questions about physics
math:Good at answer questions about math
histor

In [26]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)


In [27]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destionation_chains,
                         default_chain=default_chain,
                         verbose=True)

In [28]:
chain.run("What is the speed of light?")

/Users/sonph36/dev/MAS_Applications/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...
physics: {'input': 'What is the speed of light?'}
> Finished chain.


'The speed of light in a vacuum is approximately 299,792,458 meters per second, or about 186,282 miles per second.'

In [29]:
chain.run("What is the derivative of x^2?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is the derivative of x^2?'}
> Finished chain.


'The derivative of x^2 is 2x. This can be found using the power rule of differentiation, which states that the derivative of x^n is n*x^(n-1). In this case, n=2, so the derivative of x^2 is 2*x^(2-1) = 2x.'

In [30]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'